In [80]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from vega_datasets import data

from src import Gleaner

df = data.movies()
gleaner = Gleaner(df)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
gleaner.train_display(["count"])

Epoch 45


alt.VConcatChart(...)

In [101]:
gleaner.infer().display()

TypeError: Gleaner.infer() takes 1 positional argument but 2 were given